In [1]:
import pandas as pd
import numpy as np
import os
import json
import requests
import glob

In [2]:
df = pd.read_csv('3_combined_dataset_postproc.csv')

In [3]:
df

,IDNum,Name,State,Country
0,0,Federated University Police Officer's Association,CA,USA
1,2,Retail Delivery Drivers Local 2785,CA,USA
2,4,Ohana Eye Center Inc,HI,USA
3,5,"CMR Financial Advisors, Inc.",HI,USA
4,6,Marin Association of Public Employees,CA,USA
...,...,...,...,...
3182,4444,Spectrum,CT,USA
3183,4445,Arista,FL,USA
3184,4446,Samsara,CO,USA
3185,4447,Target,MN,USA


In [4]:
public_companies = pd.read_json('supplemental_data/company_tickers.json').transpose()

In [5]:
import pandas as pd
from fuzzywuzzy import process  # Ensure you have fuzzywuzzy installed

def process_and_save_batch(batch_df, public_companies):
    # Assuming public_companies is accessible here, either as a parameter or a global variable

    # Function to find the best match for a given name in public_companies['title']
    def best_match(name):
        if not isinstance(name, str):
            return None, 0

        companies_list = public_companies['title'].tolist()
        output = process.extractOne(name, companies_list)
        
        if output and len(output) == 2:
            match, score = output
            return (match if score > 91 else None, score)
        else:
            print(f"Unexpected output for name '{name}': {output}")
            return None, 0

    # Apply best_match to each entry in the 'Client Name' column
    batch_df['Match Results'] = batch_df['Name'].apply(best_match)
    
    # Separate the match results into 'Ticker' and 'Ticker Match Score'
    batch_df['Ticker'] = batch_df['Match Results'].apply(lambda x: x[0])
    batch_df['Ticker Match Score'] = batch_df['Match Results'].apply(lambda x: x[1])
    
    # Drop the 'Match Results' column
    batch_df.drop('Match Results', axis=1, inplace=True)
    
    # Append results to 'results.csv', creating the file if it doesn't exist
    mode = 'a' if (pd.io.common.file_exists('results.csv')) else 'w'
    batch_df.to_csv('results.csv', mode=mode, header=(not pd.io.common.file_exists('results.csv')), index=False)

# Example usage
# Ensure df and public_companies are defined
# process_and_save_batch(df, public_companies)


In [6]:
process_and_save_batch(df.head(1), public_companies)

/var/folders/g0/k7l95yd158d8hrfq1q6gxxcr0000gq/T/ipykernel_1019/2505322860.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['Match Results'] = batch_df['Name'].apply(best_match)
/var/folders/g0/k7l95yd158d8hrfq1q6gxxcr0000gq/T/ipykernel_1019/2505322860.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['Ticker'] = batch_df['Match Results'].apply(lambda x: x[0])
/var/folders/g0/k7l95yd158d8hrfq1q6gxxcr0000gq/T/ipykernel_1019/2505322860.py:27: SettingWithCopyWarning: 
A value is trying